### Data Ingestion 

In [1]:
### Document Structure 

from langchain_core.documents import Document

In [3]:
doc  = Document(
    page_content='this is the main text content I am using for RAG',
    metadata= {
        "source": "example.txt",
        "pages": 1,
        "author":"Batman",
        "date_created": "2025-01-01"
    }
)
## This is helpful in applying filters to search things by , talking of metadata 

In [5]:
## create a txt file 
import os 
os.makedirs("../data/text_files", exist_ok=True)


In [6]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [11]:
### TextLoader 
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader 
# This shenanigan is to use whatever is not deprecatedl. 


loader = TextLoader("../data/text_files/python_intro.txt", encoding= "utf-8")
#
#If a class does not define a special __str__ method, Python falls back to __repr__.

# Many libraries (including langchain) don’t bother implementing both, so str(loader) and repr(loader) return the same string.
###
document = loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [ ]:
### Directory loader 
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the dierctory 
dir_loader = DirectoryLoader(
    "../data/text_files", 
    glob= "**/*.txt",
    loader_cls = TextLoader, ### loader class to use 
    loader_kwargs = {'encoding': "utf-8"}, 
    show_progress= False
)

dir_loader.load()

[Document(metadata={'source': '..\\data\\text_files\\machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learning: Learning with labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through rewards and penalties\n\nApplications include image recognition, speech processing, and recommendation systems\n\n\n    '),
 Document(metadata={'source': '..\\data\\text_files\\python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popu

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

## load all pdfs 
dir_loader = DirectoryLoader(
    "../data/pdf", 
    glob="**/*.pdf",
    loader_cls=PyMuPDFLoader,
    show_progress=False
)

pdf_documents = dir_loader.load()


embedding and vector store db 

In [22]:
import numpy as np 
from sentence_transformers import SentenceTransformer ### model for embedding 
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity



In [29]:
class EmbeddingManager: 
    """Handles document embedding generation using SentenceTranformer"""\
    
    def __init__(self, model_name: str= "all-MiniLM-L6-v2"):
        """
        Initialise the embedding manager 

        Atrgs: 
        model_name: HigghingFace model name for sentence embedding 
        """
        self.model_name = model_name
        self.model = None  ## we willl initialise this value later 
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension : {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts 

        Args: 
            texts: List of text strings to embed 

        Returns: 
            numpy array of embeddings with shape (len(texts), embeeding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded ")
        
        print(f"Generating embeddings for {len(texts)} texts ... ")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings 
    
    def get_embedding_dimension(self) -> int: 
        """Get the embedding dimension of the model  """
        if not self.model:
            raise ValueError("Model not loaded")
        return self.model.get_sentence_embedding_dimension()


In [30]:
embedding_manager= EmbeddingManager() ## initalise 
embedding_manager 

Loading embedding model: all-MiniLM-L6-v2


d:\Code\Learning\AI\rag-intro\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ashis\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back 

Model loaded successfully. Embedding dimension : 384


### Vector Store 

In [40]:
### Vector Store 
class VectorStore: 
    """Manages document embeddings in a ChromaDB vector store """

    def __init__(self , collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Intialise the vector store 

        Args: 
            collection_name: Name of the ChromaDB colelction 
            persist_directory: Directory to persist the vector store 
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None 
        self.collection = None 
        self._initialise_store()


    def _initialise_store(self):
        """Initialise ChromaDb client and collection """
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok = True)
            self.client = chromadb.PersistentClient(path = self.persist_directory)

            # Get or create collection 
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata= {"description":"PDF document embeddings for RAG"}
            )
            print(f"Vector store initialised. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
        
        except Exception as e:
            print(f"Error initialising vector store: {e}")
            raise     

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings tot heri vector store 

        Args: 
            documents: List of LangChain documents
            embeddings: Correspoinding embeddings for the documents 
        """
        if len(documents) != len(embeddings): 
            raise ValueError("Number of documents must match number of embeddings ")
        
        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDB
        ids=[]
        metadatas = []
        documents_text=[]
        embeddings_list= []

        for i, (doc,embedding) in enumerate(zip(documents, embeddings)):
            # generate Unique ID 
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Prepare metadata 
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # document content
            documents_text.append(doc.page_content)

            # Embedding content
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas= metadatas,
                documents= documents_text
            )

            print(f"Successfully added {len(documents)} documents to vector store ")
            print(f"Total documents in collection: {self.collection.count()}")
        except Exception as e: 
            print(f"Error adding documents to vector store : {e}")
            raise 
        

In [41]:
vectorstore = VectorStore()
vectorstore 

Vector store initialised. Collection: pdf_documents
Existing documents in collection: 0


In [42]:
### convert the text to embeddings
texts= [doc.page_content for doc in chunks]
texts

NameError: name 'chunks' is not defined